In [1]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    NUM_WORKERS = 1
    PIN_MEMORY = True
    DATA_ORDER = "cw_ins"
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    
    # only flips
    MEAN_PIXEL = np.array([0.485, 0.456, 0.406],dtype=np.float32).reshape(1,1,-1)
    STD_PIXEL = np.array([0.229, 0.224, 0.225],dtype=np.float32).reshape(1,1,-1)
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    WIDTH = 224
    HEIGHT = 224
    CROP_SIZE = 224
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
train_image_dir = "/media/Data1/interns/aravind/train2017/"
# train_image_dir = "/media/Data1/interns/aravind/val2017/"
val_image_dir = "/media/Data1/interns/aravind/val2017/"
config = ProposalConfig()
model_dir = "./models/"
train_pickle = "/media/Data1/interns/aravind/re/data/train_cwid.pickle"
# train_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"
val_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"

In [4]:
# with open(train_pickle,"rb") as train_ann:
#     train_cwid = pickle.load(train_ann)
with open(val_pickle,"rb") as val_ann:
    val_cwid = pickle.load(val_ann)

In [5]:
# train_loader = model_lib.get_loader(train_cwid,config,train_image_dir)
val_loader = model_lib.get_loader(val_cwid,config,val_image_dir)

In [6]:
import torch
import torch.nn.functional as F
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
net = model_lib.MultiHGModel()

pretrained_dict = torch.load(model_dir+"full_07.pt")
net_dict = net.state_dict()

pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}
net_dict.update(pretrained_dict) 
net.load_state_dict(net_dict)

# net.vgg.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").vgg)
# net.classifier.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").classifier)
net = net.cuda()
with torch.no_grad():
    for i,data in enumerate(val_loader):
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = data
        in_file = "./results/"+str(i)+".png"
        out_file = "./results/"+str(i)+"_out.png"
        a,b,c,d= batch_images.numpy(), batch_impulses.numpy(),batch_gt_responses.numpy(),batch_bboxes.numpy()
        a = np.moveaxis(a,1,-1)
        b = np.moveaxis(b,1,-1)
        c = np.moveaxis(c,1,-1)
        d = np.moveaxis(d,1,-1)
        
        font = ImageFont.truetype("./results/Aaargh.ttf",14) 
        
        im = (a[0]*config.STD_PIXEL + config.MEAN_PIXEL)*255
        im[:,:,0] = np.where(b[0][:,:,0]>0,255,im[:,:,0])
        im[:,:,1] = np.where(b[0][:,:,0]>0,255,im[:,:,1])
        im[:,:,2] = np.where(b[0][:,:,0]>0,255,im[:,:,2])
        
        im = Image.fromarray(im.astype(np.uint8),"RGB")
        draw = ImageDraw.Draw(im)
        draw.rectangle(((0, 0), (40, 20)), fill="black")
        draw.text((5, 5), config.CLASS_NAMES[np.argmax(batch_one_hot.numpy())], font=font,fill=(255,255,255))
        im.resize((224,224),Image.BILINEAR).save(in_file, "PNG")
        
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = batch_images.cuda(),batch_impulses.cuda(),batch_gt_responses.cuda(),batch_bboxes.cuda(),batch_one_hot.cuda()
        pred_class,pred_masks = net([batch_images,batch_impulses])
        
        pred_mask = pred_masks[1]
        pred_mask = F.threshold(F.sigmoid(pred_mask),0.7,0)
        pred_mask = pred_mask.squeeze().cpu().numpy()
        pred_mask = Image.fromarray((pred_mask*255).astype(np.uint8),"L").resize((224,224),Image.BICUBIC)
        pred_mask = np.array(pred_mask)
        img = (a[0]*config.STD_PIXEL + config.MEAN_PIXEL)*255
        img[:,:,0] = np.where(pred_mask>0,pred_mask,img[:,:,0])
        im = Image.fromarray(img.astype(np.uint8),"RGB").resize((224,224),Image.BILINEAR)
        
        pred_class = F.softmax(pred_class,dim=-1).squeeze()
        maxs, indices = torch.topk(pred_class,5,-1)
        pred_class = indices[0]
        
        draw = ImageDraw.Draw(im)
        draw.rectangle(((0, 0), (40, 20)), fill="black")
        draw.text((5, 5), config.CLASS_NAMES[int(pred_class)], font=font,fill=(255,255,255))
        im.save(out_file, "PNG")

        for i in range(5):
            print(maxs[i],indices[i],config.CLASS_NAMES[int(indices[i])])
#         input()

tensor(0.9928, device='cuda:0') tensor(15, device='cuda:0') bird
tensor(1.00000e-03 *
       6.0499, device='cuda:0') tensor(26, device='cuda:0') umbrella
tensor(1.00000e-04 *
       2.6487, device='cuda:0') tensor(20, device='cuda:0') cow
tensor(1.00000e-04 *
       2.0504, device='cuda:0') tensor(19, device='cuda:0') sheep
tensor(1.00000e-04 *
       1.8911, device='cuda:0') tensor(21, device='cuda:0') elephant
tensor(0.9384, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       1.3726, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       1.2262, device='cuda:0') tensor(58, device='cuda:0') couch
tensor(1.00000e-03 *
       8.6421, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-03 *
       5.0063, device='cuda:0') tensor(61, device='cuda:0') dining table
tensor(0.9968, device='cuda:0') tensor(68, device='cuda:0') cell phone
tensor(1.00000e-03 *
       1.8622, device='cuda:0') tensor(66, device='cuda:0') remote
tensor(

tensor(0.8131, device='cuda:0') tensor(41, device='cuda:0') wine glass
tensor(1.00000e-02 *
       8.1481, device='cuda:0') tensor(42, device='cuda:0') cup
tensor(1.00000e-02 *
       3.4347, device='cuda:0') tensor(76, device='cuda:0') vase
tensor(1.00000e-02 *
       1.8936, device='cuda:0') tensor(46, device='cuda:0') bowl
tensor(1.00000e-02 *
       1.3790, device='cuda:0') tensor(40, device='cuda:0') bottle
tensor(0.9837, device='cuda:0') tensor(9, device='cuda:0') boat
tensor(1.00000e-03 *
       7.4095, device='cuda:0') tensor(6, device='cuda:0') bus
tensor(1.00000e-03 *
       5.0517, device='cuda:0') tensor(8, device='cuda:0') truck
tensor(1.00000e-03 *
       2.5118, device='cuda:0') tensor(7, device='cuda:0') train
tensor(1.00000e-04 *
       5.2420, device='cuda:0') tensor(5, device='cuda:0') airplane
tensor(0.8529, device='cuda:0') tensor(26, device='cuda:0') umbrella
tensor(0.1010, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-02 *
       1.7694, de

tensor(0.7149, device='cuda:0') tensor(55, device='cuda:0') donut
tensor(0.2001, device='cuda:0') tensor(56, device='cuda:0') cake
tensor(1.00000e-02 *
       4.2061, device='cuda:0') tensor(49, device='cuda:0') sandwich
tensor(1.00000e-02 *
       1.7952, device='cuda:0') tensor(47, device='cuda:0') banana
tensor(1.00000e-02 *
       1.0308, device='cuda:0') tensor(53, device='cuda:0') hot dog
tensor(0.9316, device='cuda:0') tensor(48, device='cuda:0') apple
tensor(1.00000e-02 *
       6.1350, device='cuda:0') tensor(50, device='cuda:0') orange
tensor(1.00000e-03 *
       3.9132, device='cuda:0') tensor(47, device='cuda:0') banana
tensor(1.00000e-03 *
       1.0238, device='cuda:0') tensor(55, device='cuda:0') donut
tensor(1.00000e-04 *
       5.7684, device='cuda:0') tensor(53, device='cuda:0') hot dog
tensor(0.2589, device='cuda:0') tensor(48, device='cuda:0') apple
tensor(0.2524, device='cuda:0') tensor(46, device='cuda:0') bowl
tensor(0.1829, device='cuda:0') tensor(47, device='cu

tensor(0.9994, device='cuda:0') tensor(38, device='cuda:0') surfboard
tensor(1.00000e-04 *
       4.9065, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-05 *
       5.3668, device='cuda:0') tensor(31, device='cuda:0') skis
tensor(1.00000e-05 *
       4.0082, device='cuda:0') tensor(9, device='cuda:0') boat
tensor(1.00000e-05 *
       1.0091, device='cuda:0') tensor(32, device='cuda:0') snowboard
tensor(0.7160, device='cuda:0') tensor(74, device='cuda:0') book
tensor(1.00000e-02 *
       6.9066, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       2.8000, device='cuda:0') tensor(61, device='cuda:0') dining table
tensor(1.00000e-02 *
       1.3001, device='cuda:0') tensor(63, device='cuda:0') tv
tensor(1.00000e-02 *
       1.2535, device='cuda:0') tensor(29, device='cuda:0') suitcase
tensor(0.7057, device='cuda:0') tensor(3, device='cuda:0') car
tensor(0.2901, device='cuda:0') tensor(8, device='cuda:0') truck
tensor(1.00000e-03 *
       3.908

tensor(0.2133, device='cuda:0') tensor(77, device='cuda:0') scissors
tensor(0.1304, device='cuda:0') tensor(44, device='cuda:0') knife
tensor(0.1117, device='cuda:0') tensor(68, device='cuda:0') cell phone
tensor(1.00000e-02 *
       8.0405, device='cuda:0') tensor(66, device='cuda:0') remote
tensor(1.00000e-02 *
       7.0584, device='cuda:0') tensor(5, device='cuda:0') airplane
tensor(0.8178, device='cuda:0') tensor(15, device='cuda:0') bird
tensor(1.00000e-02 *
       8.2596, device='cuda:0') tensor(17, device='cuda:0') dog
tensor(1.00000e-02 *
       7.6289, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-03 *
       5.5427, device='cuda:0') tensor(18, device='cuda:0') horse
tensor(1.00000e-03 *
       4.1027, device='cuda:0') tensor(36, device='cuda:0') baseball glove
tensor(0.2898, device='cuda:0') tensor(16, device='cuda:0') cat
tensor(0.2314, device='cuda:0') tensor(19, device='cuda:0') sheep
tensor(0.1738, device='cuda:0') tensor(20, device='cuda:0') cow
ten

...

tensor(0.7997, device='cuda:0') tensor(60, device='cuda:0') bed
tensor(0.1824, device='cuda:0') tensor(58, device='cuda:0') couch
tensor(1.00000e-03 *
       8.3144, device='cuda:0') tensor(61, device='cuda:0') dining table
tensor(1.00000e-03 *
       8.0416, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-04 *
       5.4805, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(0.9931, device='cuda:0') tensor(68, device='cuda:0') cell phone
tensor(1.00000e-03 *
       2.9511, device='cuda:0') tensor(66, device='cuda:0') remote
tensor(1.00000e-03 *
       2.4309, device='cuda:0') tensor(64, device='cuda:0') laptop
tensor(1.00000e-04 *
       7.9540, device='cuda:0') tensor(74, device='cuda:0') book
tensor(1.00000e-04 *
       4.4860, device='cuda:0') tensor(65, device='cuda:0') mouse
tensor(0.9938, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-03 *
       5.0060, device='cuda:0') tensor(11, device='cuda:0') fire hydrant
tensor(1.00000e-04 *
     

Process Process-1:


tensor(0.3358, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(0.1586, device='cuda:0') tensor(15, device='cuda:0') bird
tensor(1.00000e-02 *
       9.9157, device='cuda:0') tensor(9, device='cuda:0') boat
tensor(1.00000e-02 *
       4.9949, device='cuda:0') tensor(16, device='cuda:0') cat
tensor(1.00000e-02 *
       4.6928, device='cuda:0') tensor(26, device='cuda:0') umbrella
tensor(0.1441, device='cuda:0') tensor(14, device='cuda:0') bench
tensor(0.1282, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(0.1269, device='cuda:0') tensor(23, device='cuda:0') zebra
tensor(0.1189, device='cuda:0') tensor(34, device='cuda:0') kite
tensor(1.00000e-02 *
       9.2899, device='cuda:0') tensor(36, device='cuda:0') baseball glove


Traceback (most recent call last):
  File "/media/Data1/interns/aravind/anaconda3/envs/re/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/media/Data1/interns/aravind/anaconda3/envs/re/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/media/Data1/interns/aravind/anaconda3/envs/re/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/media/Data1/interns/aravind/anaconda3/envs/re/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/media/Data1/interns/aravind/re/model_lib.py", line 48, in __getitem__
    image, impulse, gt_response, one_hot, is_bad_image = self.generate_targets(image, masks, class_id, is_crowd)
  File "/media/Data1/interns/aravind/re/model_lib.py", line 157, in gene

KeyboardInterrupt: 